In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_from_disk

balanced_selfMA_Toxigen = load_from_disk("/content/drive/MyDrive/266_project/balanced_selfMA_ds_toxigen_quote")

In [ ]:
#convert entire dataset to CSV

import pandas as pd

dfs = []
for split in balanced_selfMA_Toxigen:
    df = balanced_selfMA_Toxigen[split].to_pandas()
    df["split"] = split
    dfs.append(df)

full_df = pd.concat(dfs, ignore_index=True)
full_df.to_csv("balanced_selfMA_Toxigen.csv", index=False)

In [ ]:
#read CSV
balanced_selfMA_Toxigen = pd.read_csv("/content/balanced_selfMA_Toxigen.csv")

In [ ]:
#split datset according to label

import pandas as pd

# all label 0 and label 2
df_Toxigen = balanced_selfMA_Toxigen[balanced_selfMA_Toxigen['label'].isin([0, 2])]

# Dataset B = all label 1
df_selfMA = balanced_selfMA_Toxigen[balanced_selfMA_Toxigen['label'] == 1]

print("df_Toxigen (labels 0 and 2):", df_Toxigen.shape)
print("Dataset B (label 1):", df_selfMA.shape)


df_Toxigen (labels 0 and 2): (2600, 3)
Dataset B (label 1): (1300, 3)


In [ ]:
#split toxigen into two datasets

df_Toxigen_0 = df_Toxigen[df_Toxigen['label'] == 0].copy()
df_Toxigen_2 = df_Toxigen[df_Toxigen['label'] == 2].copy()

print("Label 0 subset:", df_Toxigen_0.shape)
print("Label 2 subset:", df_Toxigen_2.shape)

Label 0 subset: (1300, 3)
Label 2 subset: (1300, 3)


In [ ]:
#sanity check
df_Toxigen_0.head()

,text,label,split
3,there is no difference in the quality of paren...,0,train
6,"it's not just about being a model minority, bu...",0,train
12,awesome,0,train
20,"because chinese new year is coming up soon, al...",0,train
22,IQ tests and related tests only measure intell...,0,train


In [ ]:
df_Toxigen_2.head()

,text,label,split
4,"Asians don't really speak proper English, do t...",2,train
10,"if you are white, you are automatically smarte...",2,train
11,they should not live past 1- we have too littl...,2,train
13,im not racist but i hate mexicans,2,train
14,explain to me why more whites do drugs than bl...,2,train


In [ ]:
#installs for readability tests

!pip install -q textstat tqdm

import numpy as np
from tqdm import tqdm
import textstat
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.8 MB/s eta 0:00:00


In [ ]:
#reset index

df_Toxigen_0 = df_Toxigen_0.reset_index(drop=True)
df_Toxigen_2  = df_Toxigen_2.reset_index(drop=True)

print(f"Rows in Toxigen 0: {len(df_Toxigen_0)}")
print(f"Rows in Rows in Toxigen 2:    {len(df_Toxigen_2)}")

Rows in Toxigen 0: 1300
Rows in Rows in Toxigen 2:    1300


In [ ]:
#readability tests

def safe_metric(func, text):
    """Run a textstat metric and return NaN if it errors."""
    try:
        return func(text)
    except Exception:
        return np.nan

def readability_metrics(text: str):
    """
    Compute readability metrics for ANY text (even short fragments).
    Returns a dict; errors in individual metrics become NaN.
    """
    text = str(text).strip()

    return {
        "flesch_reading_ease":         safe_metric(textstat.flesch_reading_ease, text),
        "flesch_kincaid_grade":        safe_metric(textstat.flesch_kincaid_grade, text),
        "gunning_fog":                 safe_metric(textstat.gunning_fog, text),
        "smog_index":                  safe_metric(textstat.smog_index, text),
        "coleman_liau_index":          safe_metric(textstat.coleman_liau_index, text),
        "automated_readability_index": safe_metric(textstat.automated_readability_index, text),
        "dale_chall_readability":      safe_metric(textstat.dale_chall_readability_score, text),
        "linsear_write":               safe_metric(textstat.linsear_write_formula, text),
        "difficult_words":             safe_metric(textstat.difficult_words, text),
        "avg_sentence_length":         safe_metric(textstat.avg_sentence_length, text),
        "avg_syllables_per_word":      safe_metric(textstat.avg_syllables_per_word, text),
        "text_standard_grade":         safe_metric(lambda t: textstat.text_standard(t, float_output=False), text),
        "word_count":                  len(text.split()) if text else 0,
        "char_count":                  len(text),
    }


def compute_readability_df(df_in: pd.DataFrame, text_col="text") -> pd.DataFrame:
    rows = []
    for t in tqdm(df_in[text_col].tolist(), desc="Computing readability"):
        rows.append(readability_metrics(t))
    return pd.DataFrame(rows)


# --- Compute metrics for label 0 and label 2 ---

read_Toxigen_0 = compute_readability_df(df_Toxigen_0, text_col="text")
read_Toxigen_2 = compute_readability_df(df_Toxigen_2, text_col="text")

# Attach metrics
df_Toxigen_0_metrics = pd.concat([df_Toxigen_0.reset_index(drop=True), read_Toxigen_0], axis=1)
df_Toxigen_2_metrics = pd.concat([df_Toxigen_2.reset_index(drop=True), read_Toxigen_2], axis=1)

# --- Compare groups: means ---

NUMERIC_COLS = [
    "flesch_reading_ease",
    "flesch_kincaid_grade",
    "gunning_fog",
    "smog_index",
    "coleman_liau_index",
    "automated_readability_index",
    "dale_chall_readability",
    "linsear_write",
    "difficult_words",
    "avg_sentence_length",
    "avg_syllables_per_word",
    "word_count",
    "char_count",
]

summary = pd.DataFrame({
    "metric": NUMERIC_COLS,
    "Toxigen_0_mean": [df_Toxigen_0_metrics[c].mean() for c in NUMERIC_COLS],
    "Toxigen_2_mean": [df_Toxigen_2_metrics[c].mean() for c in NUMERIC_COLS],
})

print("\n=== Readability Comparison (Toxigen 0 vs Toxigen 2) ===")
print(summary)




Computing readability:   0%|          | 0/1300 [00:00<?, ?it/s]/tmp/ipython-input-3136715027.py:6: DeprecationWarning: The 'avg_sentence_length' method has been deprecated due to being the same as 'words_per_sentence'. This method will be removed in thefuture.
  return func(text)
Computing readability: 100%|██████████| 1300/1300 [00:00<00:00, 3817.84it/s]


=== Readability Comparison (Toxigen 0 vs Toxigen 2) ===
                         metric  Toxigen_0_mean  Toxigen_2_mean
0           flesch_reading_ease       62.148301       71.971457
1          flesch_kincaid_grade        8.681676        7.405729
2                   gunning_fog       10.863273        9.588219
3                    smog_index        9.927566        8.315562
4            coleman_liau_index        7.445859        6.310588
5   automated_readability_index        8.251532        7.225560
6        dale_chall_readability        8.065716        7.395852
7                 linsear_write        9.828237        9.140032
8               difficult_words        3.113846        2.415385
9           avg_sentence_length       16.671987       16.845833
10       avg_syllables_per_word        1.500817        1.392022
11                   word_count       17.831538       18.343846
12                   char_count       94.905385       94.231538


In [ ]:



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix



NUMERIC_COLS = [
    "flesch_reading_ease",
    "flesch_kincaid_grade",
    "gunning_fog",
    "smog_index",
    "coleman_liau_index",
    "automated_readability_index",
    "dale_chall_readability",
    "linsear_write",
    "difficult_words",
    "avg_sentence_length",
    "avg_syllables_per_word",
    "word_count",
    "char_count",
]

# --- 1) Build X, y from the two datasets (0 vs 2) ---
X0 = df_Toxigen_0_metrics[NUMERIC_COLS].apply(pd.to_numeric, errors="coerce")
y0 = pd.Series(0, index=X0.index)

X2 = df_Toxigen_2_metrics[NUMERIC_COLS].apply(pd.to_numeric, errors="coerce")
y2 = pd.Series(1, index=X2.index)   # label "1" means: came from label=2 split

X = pd.concat([X0, X2], ignore_index=True)
y = pd.concat([y0, y2], ignore_index=True)

# Drop rows with any NaNs in features
mask_valid = ~X.isna().any(axis=1)
X = X.loc[mask_valid].reset_index(drop=True)
y = y.loc[mask_valid].reset_index(drop=True)

print("Feature matrix shape:", X.shape)
print("Class counts:\n", y.value_counts())

# --- 2) Train/test split ---
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# --- 3) Scale features (helps linear models) ---
scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr)
X_te_s = scaler.transform(X_te)

# --- 4) Train a trivial classifier (logistic regression) ---
clf = LogisticRegression(max_iter=2000, n_jobs=None)  # add class_weight="balanced" if highly imbalanced
clf.fit(X_tr_s, y_tr)

# --- 5) Evaluate ---
y_pred = clf.predict(X_te_s)
y_proba = clf.predict_proba(X_te_s)[:, 1]

acc = accuracy_score(y_te, y_pred)
auc = roc_auc_score(y_te, y_proba)
cm  = confusion_matrix(y_te, y_pred, labels=[0,1])

print("\n=== Readability-Only Classifier (Toxigen 0 vs 2) ===")
print(f"Accuracy : {acc:.4f}")
print(f"ROC-AUC  : {auc:.4f}\n")
print("Classification Report (0=Toxigen_0, 1=Toxigen_2):")
print(classification_report(y_te, y_pred, target_names=["Toxigen_0","Toxigen_2"], digits=4))
print("Confusion Matrix (rows=true, cols=pred):\n", cm)

# --- 6) (Optional) Inspect which features push toward label=2 ---
coef_df = pd.DataFrame({"feature": NUMERIC_COLS, "weight": clf.coef_[0]}).sort_values("weight", ascending=False)
print("\nTop weights (positive → predicts Toxigen_2):")
print(coef_df)


Feature matrix shape: (2600, 13)
Class counts:
 0    1300
1    1300
Name: count, dtype: int64

=== Readability-Only Classifier (Toxigen 0 vs 2) ===
Accuracy : 0.6167
ROC-AUC  : 0.6510

Classification Report (0=Toxigen_0, 1=Toxigen_2):
              precision    recall  f1-score   support

   Toxigen_0     0.6213    0.5974    0.6092       390
   Toxigen_2     0.6123    0.6359    0.6239       390

    accuracy                         0.6167       780
   macro avg     0.6168    0.6167    0.6165       780
weighted avg     0.6168    0.6167    0.6165       780

Confusion Matrix (rows=true, cols=pred):
 [[233 157]
 [142 248]]

Top weights (positive → predicts Toxigen_2):
                        feature    weight
9           avg_sentence_length  1.459604
12                   char_count  0.970186
0           flesch_reading_ease  0.599777
1          flesch_kincaid_grade  0.560557
4            coleman_liau_index  0.260233
2                   gunning_fog  0.252396
6        dale_chall_readability  